In [103]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timedelta
import numpy as np

## Cargar los datos

In [104]:
datos_electricos = pd.read_csv("data_proyecto.csv", encoding= "utf-8")
datos_electricos["energy(Wh)"] = datos_electricos["energy(Wh)"]/1000
datos_electricos["power(W)"] = datos_electricos["power(W)"]/1000
datos_electricos = datos_electricos.rename(columns = {"power(W)":"power(kW)","energy(Wh)":"energy(kWh)"})

datos_electricos.head()

,device_id,device_name,measurement_time(UTC),current(A),voltage(V),power(kW),power_factor,energy(kWh)
0,82608,Linea 2 Tablero Principal,1/1/2019 0:00,0.71,135.3,0.0931,0.97,0.0233
1,73170,L3 Transportador Botellas PDC III,1/1/2019 0:00,2.94,127.0,0.3551,0.95,0.0888
2,72819,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,6.34,127.0,0.7644,0.95,0.1911
3,72823,L4 Control de Inyectores,1/1/2019 0:00,30.97,127.0,3.7360,0.95,0.9340
4,82649,L4 Mezcladora Paramix,1/1/2019 0:00,4.79,127.0,0.5780,0.95,0.1445


In [105]:
datos_grupos = pd.read_excel("Device_Groups.xlsx")
datos_grupos.head()

,device_name,DG1,DG2,Usable
0,Alumbrado General L1 y Tratamiento Aguas,Potencia Industrial,Otros,0
1,Compresor 1 Aire 250 HP,Potencia Industrial,Compresores de aire,1
2,Compresor 2 Aire 200 HP,Potencia Industrial,Compresores de aire,1
3,Compresor de Aire 1 60 HP,Potencia Industrial,Compresores de aire,1
4,Compresor de Aire 2 60 HP,Potencia Industrial,Compresores de aire,1


In [106]:
datos_produccion = pd.read_excel("produccion_lineas.xlsx")
fecha_formato = [datetime.strptime(date,"%d/%m/%Y") for date in datos_produccion["Fecha"].values.tolist()]
datos_produccion["Fecha"] = fecha_formato
datos_produccion["Fecha"] = datos_produccion["Fecha"].dt.date
datos_produccion.head()

,Fecha,Prod. Linea 1(LTS),Prod. Linea 2(LTS),Prod. Linea 3(LTS),Prod. Linea 4(LTS)
0,2019-07-31,0,0.0,0,0
1,2019-07-30,0,0.0,0,0
2,2019-07-29,0,0.0,0,0
3,2019-07-28,0,0.0,0,0
4,2019-07-27,0,0.0,0,0


In [107]:
datos_electricos2 = datos_electricos.drop(columns=["device_id","current(A)","voltage(V)","power_factor","energy(kWh)"])
datos_electricos2.head()

,device_name,measurement_time(UTC),power(kW)
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0931
1,L3 Transportador Botellas PDC III,1/1/2019 0:00,0.3551
2,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,0.7644
3,L4 Control de Inyectores,1/1/2019 0:00,3.7360
4,L4 Mezcladora Paramix,1/1/2019 0:00,0.5780


In [108]:
datos_electricos3=pd.merge(left = datos_electricos2, right = datos_grupos, on="device_name", how = "left").drop(columns="DG1")
datos_electricos3.head()

,device_name,measurement_time(UTC),power(kW),DG2,Usable
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0931,Linea 2 Total,1.0
1,L3 Transportador Botellas PDC III,1/1/2019 0:00,0.3551,Linea 3,0.0
2,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,0.7644,Otros,0.0
3,L4 Control de Inyectores,1/1/2019 0:00,3.7360,Linea 4 Total,1.0
4,L4 Mezcladora Paramix,1/1/2019 0:00,0.5780,NaN,NaN


## MATRIZ

In [109]:
# Obtengo un primer DataFrame para los datos de la matriz
datos_matriz = datos_electricos3[(datos_electricos3["Usable"]==1 )].drop(columns= ["Usable","DG2"])
datos_matriz.head()

,device_name,measurement_time(UTC),power(kW)
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0931
3,L4 Control de Inyectores,1/1/2019 0:00,3.7360
15,L4 PDC II,1/1/2019 0:00,4.2513
16,Compresor de Amoniaco 2 Tornillo 250 HP VFD,1/1/2019 0:00,0.4013
17,Compresor de Aire 1 60 HP,1/1/2019 0:00,0.0072


In [110]:
#Cargo datos de umbrales de potencia y pivoteo los datos de la matriz
umbrales = pd.read_excel("Umbrales.xlsx")
datos_matriz = pd.pivot_table(datos_matriz,index="measurement_time(UTC)", columns = "device_name", values="power(kW)",fill_value=0)
datos_matriz.head()

device_name,Compresor 1 Aire 250 HP,Compresor 2 Aire 200 HP,Compresor de Aire 1 60 HP,Compresor de Aire 2 60 HP,Compresor de Aire 3 60 HP,Compresor de Amoniaco 1 Tornillo 250 HP,Compresor de Amoniaco 2 Tornillo 250 HP VFD,Compresor de Amoniaco 3 Tornillo 250 HP VFD,Compresor de Amoniaco 4 Tornillo 250 HP,Compresor de Amoniaco 5 Tornillo 250 HP,...,L4 Control de Inyectores,L4 PDC I,L4 PDC II,L4 PDC IV,Linea 1 Tablero Principal,Linea 2 Tablero Principal,Linea 3 Tablero PDC I,Linea 3 Tablero PDC II,Linea 3 Tablero PDC III,Linea 3 Tablero PDC IV
measurement_time(UTC),,,,,,,,,,,,,,,,,,,,,
1/1/2019 0:00,0.0000,0.1835,0.0072,0.0408,0.0,0.0,0.4013,0.3847,0.0,0.0,...,3.7360,2.9255,4.2513,2.3617,5.5157,0.0931,4.1830,3.0994,0.3642,0.5788
1/1/2019 0:15,0.0000,0.1802,0.0035,0.0408,0.0,0.0,0.3919,0.3796,0.0,0.0,...,3.7426,2.9638,4.2383,2.3564,5.5105,0.1650,4.1915,3.0235,0.3640,0.5817
1/1/2019 0:30,0.0000,0.1827,0.0000,0.0426,0.0,0.0,0.3974,0.3839,0.0,0.0,...,3.7270,2.9859,4.2402,2.3479,5.5107,0.0428,4.1783,3.0505,0.3634,0.5683
1/1/2019 0:45,0.0000,0.1810,0.0012,0.0421,0.0,0.0,0.3984,0.3803,0.0,0.0,...,3.7353,2.9888,4.2218,2.3510,5.4984,0.0580,4.1809,3.0663,0.3638,0.5699
1/1/2019 10:15,0.0025,107.9400,0.0296,0.0375,0.0,0.0,0.3856,0.3735,0.0,0.0,...,3.7129,4.8533,2.3989,2.4343,12.5873,1.4543,3.9350,3.3506,0.3629,0.7058


In [111]:
#Creo nueva columnas para los totales de las lineas
datos_matriz["Linea 1 Total"] = datos_matriz["Linea 1 Tablero Principal"] + datos_matriz["L1 Tablero de Controles Mezclador"]
datos_matriz["Linea 2 Total"] = datos_matriz["Linea 2 Tablero Principal"]
datos_matriz["Linea 3 Total"] = datos_matriz["Linea 3 Tablero PDC I"] +datos_matriz["Linea 3 Tablero PDC II"]+datos_matriz["Linea 3 Tablero PDC III"] + datos_matriz["Linea 3 Tablero PDC IV"]
datos_matriz["Linea 4 Total"] = datos_matriz["L4 PDC I"] + datos_matriz["L4 PDC II"] + datos_matriz["L4 PDC IV"]


#Elimino los equipos que conforman los totales de las lineas
datos_matriz = datos_matriz.drop(columns=["Linea 1 Tablero Principal", "L1 Tablero de Controles Mezclador", "Linea 2 Tablero Principal",
                                         "Linea 3 Tablero PDC I","Linea 3 Tablero PDC II","Linea 3 Tablero PDC III","Linea 3 Tablero PDC IV",
                                         "L4 PDC I","L4 PDC II","L4 PDC IV"])
#Obtengo la lista de columnas que ser'an las value_vars al hacer el melt
lista_columnas=datos_matriz.columns.tolist()

#Hago que measurement_time vuelva a ser columna y no indice
datos_matriz= datos_matriz.reset_index()
datos_matriz.head()

device_name,measurement_time(UTC),Compresor 1 Aire 250 HP,Compresor 2 Aire 200 HP,Compresor de Aire 1 60 HP,Compresor de Aire 2 60 HP,Compresor de Aire 3 60 HP,Compresor de Amoniaco 1 Tornillo 250 HP,Compresor de Amoniaco 2 Tornillo 250 HP VFD,Compresor de Amoniaco 3 Tornillo 250 HP VFD,Compresor de Amoniaco 4 Tornillo 250 HP,Compresor de Amoniaco 5 Tornillo 250 HP,Compresor de Amoniaco 6 Tornillo 250 HP VFD,Compresor de Amoniaco 7 Tornillo 250 HP,Compresor de Amoniaco Reciprocante 1 200 HP,Compresor de Amoniaco Reciprocante 2 200 HP,L4 Control de Inyectores,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total
0,1/1/2019 0:00,0.0000,0.1835,0.0072,0.0408,0.0,0.0,0.4013,0.3847,0.0,0.0,0.0000,0.0,0.0,0.0,3.7360,14.7085,0.0931,8.2254,9.5385
1,1/1/2019 0:15,0.0000,0.1802,0.0035,0.0408,0.0,0.0,0.3919,0.3796,0.0,0.0,0.0000,0.0,0.0,0.0,3.7426,14.6277,0.1650,8.1607,9.5585
2,1/1/2019 0:30,0.0000,0.1827,0.0000,0.0426,0.0,0.0,0.3974,0.3839,0.0,0.0,0.0000,0.0,0.0,0.0,3.7270,14.5738,0.0428,8.1605,9.5740
3,1/1/2019 0:45,0.0000,0.1810,0.0012,0.0421,0.0,0.0,0.3984,0.3803,0.0,0.0,0.0001,0.0,0.0,0.0,3.7353,14.6034,0.0580,8.1809,9.5616
4,1/1/2019 10:15,0.0025,107.9400,0.0296,0.0375,0.0,0.0,0.3856,0.3735,0.0,0.0,0.0000,0.0,0.0,0.0,3.7129,21.6767,1.4543,8.3543,9.6865


In [112]:
#Hago unpivot a las columnas, dejando a measurement_time como id_var
datos_matriz=pd.melt(datos_matriz,id_vars="measurement_time(UTC)", value_vars = lista_columnas,
       var_name="device_name",value_name="power(kW)")

#Junto con el DataFrame de umbrales
datos_matriz = pd.merge(left=datos_matriz, right=umbrales, on="device_name", how="inner")
datos_matriz.head()

,measurement_time(UTC),device_name,power(kW),Umbral,Abreviacion
0,1/1/2019 0:00,Compresor 1 Aire 250 HP,0.0000,5,CAGD1
1,1/1/2019 0:15,Compresor 1 Aire 250 HP,0.0000,5,CAGD1
2,1/1/2019 0:30,Compresor 1 Aire 250 HP,0.0000,5,CAGD1
3,1/1/2019 0:45,Compresor 1 Aire 250 HP,0.0000,5,CAGD1
4,1/1/2019 10:15,Compresor 1 Aire 250 HP,0.0025,5,CAGD1


In [113]:
# Creo columna condicional que determina el estado del equipo
datos_matriz["Estado"] = np.where(datos_matriz["power(kW)"]>= datos_matriz["Umbral"], datos_matriz["Abreviacion"],"")

#Tiro columnas de umbrales y abreviaciones
datos_matriz = datos_matriz.drop(columns=["Umbral", "Abreviacion"])
datos_matriz.head()

,measurement_time(UTC),device_name,power(kW),Estado
0,1/1/2019 0:00,Compresor 1 Aire 250 HP,0.0000,
1,1/1/2019 0:15,Compresor 1 Aire 250 HP,0.0000,
2,1/1/2019 0:30,Compresor 1 Aire 250 HP,0.0000,
3,1/1/2019 0:45,Compresor 1 Aire 250 HP,0.0000,
4,1/1/2019 10:15,Compresor 1 Aire 250 HP,0.0025,


#### Matriz  de estados

In [114]:

#Creo nuevo DataFrame que solo guarde los datos de estado, pero no los de potencia
matriz_estados = datos_matriz.drop(columns="power(kW)")
matriz_estados = pd.pivot_table(matriz_estados, index="measurement_time(UTC)", columns ="device_name", values="Estado", aggfunc="sum")
matriz_estados.head()

device_name,Compresor 1 Aire 250 HP,Compresor 2 Aire 200 HP,Compresor de Aire 1 60 HP,Compresor de Aire 2 60 HP,Compresor de Aire 3 60 HP,Compresor de Amoniaco 1 Tornillo 250 HP,Compresor de Amoniaco 2 Tornillo 250 HP VFD,Compresor de Amoniaco 3 Tornillo 250 HP VFD,Compresor de Amoniaco 4 Tornillo 250 HP,Compresor de Amoniaco 5 Tornillo 250 HP,Compresor de Amoniaco 6 Tornillo 250 HP VFD,Compresor de Amoniaco 7 Tornillo 250 HP,Compresor de Amoniaco Reciprocante 1 200 HP,Compresor de Amoniaco Reciprocante 2 200 HP,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total
measurement_time(UTC),,,,,,,,,,,,,,,,,,
1/1/2019 0:00,,,,,,,,,,,,,,,,,,
1/1/2019 0:15,,,,,,,,,,,,,,,,,,
1/1/2019 0:30,,,,,,,,,,,,,,,,,,
1/1/2019 0:45,,,,,,,,,,,,,,,,,,
1/1/2019 10:15,,CAGD2,,,,,,,,,,,,,,,,


In [115]:
#Genero columnas con las cargas de compresores y lineas
matriz_estados["CargaRefL1L3L4"] = (matriz_estados["Compresor de Amoniaco 1 Tornillo 250 HP"] + 
                                    matriz_estados["Compresor de Amoniaco 2 Tornillo 250 HP VFD"]+
                                    matriz_estados["Compresor de Amoniaco 3 Tornillo 250 HP VFD"]+
                                    matriz_estados["Compresor de Amoniaco 4 Tornillo 250 HP"]+
                                    matriz_estados["Compresor de Amoniaco 5 Tornillo 250 HP"]+
                                    matriz_estados["Compresor de Amoniaco Reciprocante 2 200 HP"]
                                   )
matriz_estados["CargaRefL2"] = (matriz_estados["Compresor de Amoniaco 6 Tornillo 250 HP VFD"]+
                                matriz_estados["Compresor de Amoniaco 7 Tornillo 250 HP"]+
                                matriz_estados["Compresor de Amoniaco Reciprocante 1 200 HP"]
                                )

matriz_estados["CargaCompresoresAire"]=(matriz_estados["Compresor 1 Aire 250 HP"]+
                                        matriz_estados["Compresor 2 Aire 200 HP"]+
                                        matriz_estados["Compresor de Aire 1 60 HP"]+
                                        matriz_estados["Compresor de Aire 2 60 HP"]+
                                        matriz_estados["Compresor de Aire 3 60 HP"]
                                       )

matriz_estados["CargaLineasL1L3L4"] = (matriz_estados["Linea 1 Total"]+
                                       matriz_estados["Linea 3 Total"]+
                                       matriz_estados["Linea 4 Total"]
                                      )

matriz_estados["CargaLineasL2"] = matriz_estados["Linea 2 Total"]

matriz_estados["CargaLineasL1L2L3L4"] = (matriz_estados["Linea 1 Total"]+
                                         matriz_estados["Linea 2 Total"]+
                                         matriz_estados["Linea 3 Total"]+
                                         matriz_estados["Linea 4 Total"]
                                      )
matriz_estados.tail()

device_name,Compresor 1 Aire 250 HP,Compresor 2 Aire 200 HP,Compresor de Aire 1 60 HP,Compresor de Aire 2 60 HP,Compresor de Aire 3 60 HP,Compresor de Amoniaco 1 Tornillo 250 HP,Compresor de Amoniaco 2 Tornillo 250 HP VFD,Compresor de Amoniaco 3 Tornillo 250 HP VFD,Compresor de Amoniaco 4 Tornillo 250 HP,Compresor de Amoniaco 5 Tornillo 250 HP,...,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total,CargaRefL1L3L4,CargaRefL2,CargaCompresoresAire,CargaLineasL1L3L4,CargaLineasL2,CargaLineasL1L2L3L4
measurement_time(UTC),,,,,,,,,,,,,,,,,,,,,
9/7/2019 8:45,CAGD1,CAGD2,,,,CRT1,CRT2,,,,...,,L2,,L4,CRT1CRT2,CRT7,CAGD1CAGD2,L4,L2,L2L4
9/7/2019 9:00,CAGD1,CAGD2,,,,CRT1,CRT2,,,,...,L1,L2,,L4,CRT1CRT2,CRT7,CAGD1CAGD2,L1L4,L2,L1L2L4
9/7/2019 9:15,CAGD1,CAGD2,,,,CRT1,CRT2,,,,...,L1,L2,,L4,CRT1CRT2,CRT7,CAGD1CAGD2,L1L4,L2,L1L2L4
9/7/2019 9:30,CAGD1,CAGD2,,,,CRT1,CRT2,,,,...,,L2,,L4,CRT1CRT2,CRT7,CAGD1CAGD2,L4,L2,L2L4
9/7/2019 9:45,CAGD1,CAGD2,,,,CRT1,CRT2,,,,...,,L2,,L4,CRT1CRT2,CRT7,CAGD1CAGD2,L4,L2,L2L4


In [116]:
matriz_estados = matriz_estados.iloc[:,[-1,-2,-3,-4,-5,-6]].reset_index()
matriz_estados.tail()

device_name,measurement_time(UTC),CargaLineasL1L2L3L4,CargaLineasL2,CargaLineasL1L3L4,CargaCompresoresAire,CargaRefL2,CargaRefL1L3L4
12436,9/7/2019 8:45,L2L4,L2,L4,CAGD1CAGD2,CRT7,CRT1CRT2
12437,9/7/2019 9:00,L1L2L4,L2,L1L4,CAGD1CAGD2,CRT7,CRT1CRT2
12438,9/7/2019 9:15,L1L2L4,L2,L1L4,CAGD1CAGD2,CRT7,CRT1CRT2
12439,9/7/2019 9:30,L2L4,L2,L4,CAGD1CAGD2,CRT7,CRT1CRT2
12440,9/7/2019 9:45,L2L4,L2,L4,CAGD1CAGD2,CRT7,CRT1CRT2


#### Matriz de potencia

In [117]:
#Creo nuevo DataFrame que solo guarde los datos de potencia, pero no los de los estados
matriz_potencia = datos_matriz.drop(columns="Estado")

matriz_potencia = pd.pivot_table(matriz_potencia, index="measurement_time(UTC)", columns ="device_name", values="power(kW)", aggfunc="sum")
matriz_potencia.head()

device_name,Compresor 1 Aire 250 HP,Compresor 2 Aire 200 HP,Compresor de Aire 1 60 HP,Compresor de Aire 2 60 HP,Compresor de Aire 3 60 HP,Compresor de Amoniaco 1 Tornillo 250 HP,Compresor de Amoniaco 2 Tornillo 250 HP VFD,Compresor de Amoniaco 3 Tornillo 250 HP VFD,Compresor de Amoniaco 4 Tornillo 250 HP,Compresor de Amoniaco 5 Tornillo 250 HP,Compresor de Amoniaco 6 Tornillo 250 HP VFD,Compresor de Amoniaco 7 Tornillo 250 HP,Compresor de Amoniaco Reciprocante 1 200 HP,Compresor de Amoniaco Reciprocante 2 200 HP,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total
measurement_time(UTC),,,,,,,,,,,,,,,,,,
1/1/2019 0:00,0.0000,0.1835,0.0072,0.0408,0.0,0.0,0.4013,0.3847,0.0,0.0,0.0000,0.0,0.0,0.0,14.7085,0.0931,8.2254,9.5385
1/1/2019 0:15,0.0000,0.1802,0.0035,0.0408,0.0,0.0,0.3919,0.3796,0.0,0.0,0.0000,0.0,0.0,0.0,14.6277,0.1650,8.1607,9.5585
1/1/2019 0:30,0.0000,0.1827,0.0000,0.0426,0.0,0.0,0.3974,0.3839,0.0,0.0,0.0000,0.0,0.0,0.0,14.5738,0.0428,8.1605,9.5740
1/1/2019 0:45,0.0000,0.1810,0.0012,0.0421,0.0,0.0,0.3984,0.3803,0.0,0.0,0.0001,0.0,0.0,0.0,14.6034,0.0580,8.1809,9.5616
1/1/2019 10:15,0.0025,107.9400,0.0296,0.0375,0.0,0.0,0.3856,0.3735,0.0,0.0,0.0000,0.0,0.0,0.0,21.6767,1.4543,8.3543,9.6865


In [118]:
#Creo nuevas columnas para los totales de los compresores
matriz_potencia["RefrigeracionL1L3L4"] =(matriz_potencia["Compresor de Amoniaco 1 Tornillo 250 HP"]+
                                      matriz_potencia["Compresor de Amoniaco 2 Tornillo 250 HP VFD"]+
                                      matriz_potencia["Compresor de Amoniaco 3 Tornillo 250 HP VFD"]+
                                      matriz_potencia["Compresor de Amoniaco 4 Tornillo 250 HP"]+
                                      matriz_potencia["Compresor de Amoniaco 5 Tornillo 250 HP"]+
                                      matriz_potencia["Compresor de Amoniaco Reciprocante 2 200 HP"]
                                      )
matriz_potencia["RefrigeracionL2"] = (matriz_potencia["Compresor de Amoniaco 6 Tornillo 250 HP VFD"]+
                                   matriz_potencia["Compresor de Amoniaco 7 Tornillo 250 HP"]+
                                   matriz_potencia["Compresor de Amoniaco Reciprocante 1 200 HP"]
                                  )
matriz_potencia["Compresores Aire"] = (matriz_potencia["Compresor 1 Aire 250 HP"]+
                                    matriz_potencia["Compresor 2 Aire 200 HP"]+
                                    matriz_potencia["Compresor de Aire 1 60 HP"]+
                                    matriz_potencia["Compresor de Aire 2 60 HP"]+
                                    matriz_potencia["Compresor de Aire 3 60 HP"]
                                   )

matriz_potencia.head()

device_name,Compresor 1 Aire 250 HP,Compresor 2 Aire 200 HP,Compresor de Aire 1 60 HP,Compresor de Aire 2 60 HP,Compresor de Aire 3 60 HP,Compresor de Amoniaco 1 Tornillo 250 HP,Compresor de Amoniaco 2 Tornillo 250 HP VFD,Compresor de Amoniaco 3 Tornillo 250 HP VFD,Compresor de Amoniaco 4 Tornillo 250 HP,Compresor de Amoniaco 5 Tornillo 250 HP,...,Compresor de Amoniaco 7 Tornillo 250 HP,Compresor de Amoniaco Reciprocante 1 200 HP,Compresor de Amoniaco Reciprocante 2 200 HP,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total,RefrigeracionL1L3L4,RefrigeracionL2,Compresores Aire
measurement_time(UTC),,,,,,,,,,,,,,,,,,,,,
1/1/2019 0:00,0.0000,0.1835,0.0072,0.0408,0.0,0.0,0.4013,0.3847,0.0,0.0,...,0.0,0.0,0.0,14.7085,0.0931,8.2254,9.5385,0.7860,0.0000,0.2315
1/1/2019 0:15,0.0000,0.1802,0.0035,0.0408,0.0,0.0,0.3919,0.3796,0.0,0.0,...,0.0,0.0,0.0,14.6277,0.1650,8.1607,9.5585,0.7715,0.0000,0.2245
1/1/2019 0:30,0.0000,0.1827,0.0000,0.0426,0.0,0.0,0.3974,0.3839,0.0,0.0,...,0.0,0.0,0.0,14.5738,0.0428,8.1605,9.5740,0.7813,0.0000,0.2253
1/1/2019 0:45,0.0000,0.1810,0.0012,0.0421,0.0,0.0,0.3984,0.3803,0.0,0.0,...,0.0,0.0,0.0,14.6034,0.0580,8.1809,9.5616,0.7787,0.0001,0.2243
1/1/2019 10:15,0.0025,107.9400,0.0296,0.0375,0.0,0.0,0.3856,0.3735,0.0,0.0,...,0.0,0.0,0.0,21.6767,1.4543,8.3543,9.6865,0.7591,0.0000,108.0096


In [119]:
matriz_potencia = matriz_potencia.iloc[:,[-1,-2,-3,-4,-5,-6,-7]].reset_index()
matriz_potencia.head()

device_name,measurement_time(UTC),Compresores Aire,RefrigeracionL2,RefrigeracionL1L3L4,Linea 4 Total,Linea 3 Total,Linea 2 Total,Linea 1 Total
0,1/1/2019 0:00,0.2315,0.0000,0.7860,9.5385,8.2254,0.0931,14.7085
1,1/1/2019 0:15,0.2245,0.0000,0.7715,9.5585,8.1607,0.1650,14.6277
2,1/1/2019 0:30,0.2253,0.0000,0.7813,9.5740,8.1605,0.0428,14.5738
3,1/1/2019 0:45,0.2243,0.0001,0.7787,9.5616,8.1809,0.0580,14.6034
4,1/1/2019 10:15,108.0096,0.0000,0.7591,9.6865,8.3543,1.4543,21.6767


In [120]:
matriz_potencia = matriz_potencia.iloc[:,[0,1,2,3]]
matriz_potencia.head()

device_name,measurement_time(UTC),Compresores Aire,RefrigeracionL2,RefrigeracionL1L3L4
0,1/1/2019 0:00,0.2315,0.0000,0.7860
1,1/1/2019 0:15,0.2245,0.0000,0.7715
2,1/1/2019 0:30,0.2253,0.0000,0.7813
3,1/1/2019 0:45,0.2243,0.0001,0.7787
4,1/1/2019 10:15,108.0096,0.0000,0.7591


#### Combinar potencia con estados

In [132]:
compresores_aire = pd.merge(left=matriz_estados, right = matriz_potencia, on="measurement_time(UTC)")[["measurement_time(UTC)",
                                                                                    "CargaLineasL1L2L3L4",
                                                                                    "CargaCompresoresAire",
                                                                                    "Compresores Aire"]]

refrigreacionL1L3L4 =pd.merge(left=matriz_estados, right = matriz_potencia, on="measurement_time(UTC)")[["measurement_time(UTC)",
                                                                                    "CargaLineasL1L3L4",
                                                                                    "CargaRefL1L3L4",
                                                                                    "RefrigeracionL1L3L4"]]

refrigeracionL2= pd.merge(left=matriz_estados, right = matriz_potencia, on="measurement_time(UTC)")[["measurement_time(UTC)",
                                                                                    "CargaLineasL2",
                                                                                    "CargaRefL2",
                                                                                    "RefrigeracionL2"]]

### Refrigeracion L1L3L4

In [190]:
refrigeracionL1L3L4V2= refrigreacionL1L3L4.copy()
#df.groupby('Name')['Credit'].agg(['sum','average'])
refrigeracionL1L3L4V2["Contador"] = 0.25
refrigeracionL1L3L4V2 = refrigeracionL1L3L4V2.groupby(['CargaLineasL1L3L4','CargaRefL1L3L4'])[["RefrigeracionL1L3L4","Contador"]].agg(["mean","sum"]).iloc[:,[0,3]].reset_index()
refrigeracionL1L3L4V2= refrigeracionL1L3L4V2[(refrigeracionL1L3L4V2["CargaLineasL1L3L4"]!="") & (refrigeracionL1L3L4V2["CargaRefL1L3L4"]!="")]
promedio_refrigeracionL1L3L4 = refrigeracionL1L3L4V2["Contador"]["sum"].mean()
refrigeracionL1L3L4V2=refrigeracionL1L3L4V2[refrigeracionL1L3L4V2["Contador"]["sum"]>=promedio_refrigeracionL1L3L4].sort_values(by=("Contador","sum"),ascending=False)
refrigeracionL1L3L4V2.head()
#.to_excel("lol2" +".xlsx")

,CargaLineasL1L3L4,CargaRefL1L3L4,RefrigeracionL1L3L4,Contador
,,,mean,sum
93,L1L3L4,CRT1CRT2CRT3,430.767062,217.75
110,L1L3L4,CRT1CRT3CRT5,451.438979,150.75
125,L1L3L4,CRT2CRT3CRT5,426.297527,144.75
172,L1L4,CRT2CRT3,268.068156,135.25
252,L3L4,CRT2CRT3,215.865297,125.00


### Refrigracion L2

In [197]:
refrigeracionL2V2= refrigeracionL2.copy()
#df.groupby('Name')['Credit'].agg(['sum','average'])
refrigeracionL2V2["Contador"] = 0.25
refrigeracionL2V2 = refrigeracionL2V2.groupby(['CargaLineasL2','CargaRefL2'])[["RefrigeracionL2","Contador"]].agg(["mean","sum"]).iloc[:,[0,3]].reset_index()
refrigeracionL2V2= refrigeracionL2V2[(refrigeracionL2V2["CargaLineasL2"]!="") & (refrigeracionL2V2["CargaRefL2"]!="")]
refrigeracionL2V2
#.to_excel("lol2" +".xlsx")

,CargaLineasL2,CargaRefL2,RefrigeracionL2,Contador
,,,mean,sum
9,L2,CRR1,34.437738,21.00
10,L2,CRT6,115.764160,98.75
11,L2,CRT6CRR1,188.200666,1273.25
12,L2,CRT6CRT7,204.050145,20.75
13,L2,CRT6CRT7CRR1,167.152489,38.50
14,L2,CRT7,148.683364,481.00
15,L2,CRT7CRR1,190.023100,504.25


### Compresores de aire

In [201]:
compresores_aireV2= compresores_aire.copy()
#df.groupby('Name')['Credit'].agg(['sum','average'])
compresores_aireV2["Contador"] = 0.25
compresores_aireV2 = compresores_aireV2.groupby(['CargaLineasL1L2L3L4','CargaCompresoresAire'])[["Compresores Aire","Contador"]].agg(["mean","sum"]).iloc[:,[0,3]].reset_index()
compresores_aireV2= compresores_aireV2[(compresores_aireV2["CargaLineasL1L2L3L4"]!="") & (compresores_aireV2["CargaCompresoresAire"]!="")]
promedio_compresores_aire = compresores_aireV2["Contador"]["sum"].mean()
compresores_aireV2=compresores_aireV2[compresores_aireV2["Contador"]["sum"]>=promedio_compresores_aire].sort_values(by=("Contador","sum"),ascending=False)
compresores_aireV2.head()
#.to_excel("lol2" +".xlsx")

,CargaLineasL1L2L3L4,CargaCompresoresAire,Compresores Aire,Contador
,,,mean,sum
63,L1L2L3L4,CAGD1CAGD2,259.545371,331.50
189,L2L3L4,CAGD1CAGD2,248.577287,164.50
85,L1L2L4,CAGD1CAGD2,236.852433,149.00
74,L1L2L3L4,CAGD1CAIR1CAIR3,260.553548,116.50
64,L1L2L3L4,CAGD1CAGD2CAIR1,284.415175,97.75


### Obtener datos para regresiones

In [182]:
datos_electricos4 = datos_electricos.drop(columns=["device_id","current(A)","voltage(V)","power_factor","power(kW)"])
datos_electricos4 = pd.merge(left = datos_electricos4, right = datos_grupos, on="device_name", how = "left").drop(columns="DG1")
datos_electricos4 =datos_electricos4[datos_electricos4["Usable"]==1].drop(columns="Usable")
datos_electricos4.head()

,device_name,measurement_time(UTC),energy(kWh),DG2
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0233,Linea 2 Total
3,L4 Control de Inyectores,1/1/2019 0:00,0.9340,Linea 4 Total
15,L4 PDC II,1/1/2019 0:00,1.0628,Linea 4 Total
16,Compresor de Amoniaco 2 Tornillo 250 HP VFD,1/1/2019 0:00,0.1003,Refrigeración L1L3L4
17,Compresor de Aire 1 60 HP,1/1/2019 0:00,0.0018,Compresores de aire


In [183]:
fecha_formato = [datetime.strptime(date,"%d/%m/%Y %H:%M") for date in datos_electricos4["measurement_time(UTC)"].values.tolist()]
datos_electricos4["measurement_time(UTC)"] = fecha_formato

In [184]:
datos_electricos4 = datos_electricos4[datos_electricos4["measurement_time(UTC)"] < datetime(2019,7,1,0,0)]
datos_electricos4.head()

,device_name,measurement_time(UTC),energy(kWh),DG2
0,Linea 2 Tablero Principal,2019-01-01,0.0233,Linea 2 Total
3,L4 Control de Inyectores,2019-01-01,0.9340,Linea 4 Total
15,L4 PDC II,2019-01-01,1.0628,Linea 4 Total
16,Compresor de Amoniaco 2 Tornillo 250 HP VFD,2019-01-01,0.1003,Refrigeración L1L3L4
17,Compresor de Aire 1 60 HP,2019-01-01,0.0018,Compresores de aire


In [185]:
datos_electricos4["Fecha"] = datos_electricos4["measurement_time(UTC)"].dt.date
datos_electricos_regresion = pd.pivot_table(datos_electricos4.groupby(["Fecha","DG2"]).sum().reset_index(),
                                            index="Fecha", columns="DG2", values="energy(kWh)",
                                           aggfunc="sum").reset_index()
datos_electricos_regresion.head()

DG2,Fecha,Compresores de aire,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total,Refrigeración L1L3L4,Refrigeración L2
0,2019-01-01,1214.4367,1281.9504,87.7999,167.8932,624.6109,1019.0041,28.3890
1,2019-01-02,4610.1620,4391.1099,883.5941,843.6383,3473.6623,5406.4273,453.9931
2,2019-01-03,4646.8085,3345.8780,1850.9111,2477.3533,2907.2174,5096.9409,2516.4700
3,2019-01-04,5128.9255,2959.1534,2102.0838,2815.7259,3600.7994,5433.8828,3000.9898
4,2019-01-05,4565.3341,3393.3607,465.6202,2394.8061,3003.6845,5007.7196,1241.6444


In [186]:
pd.merge(left=datos_electricos_regresion, right = datos_produccion, left_on="Fecha", right_on="Fecha")


,Fecha,Compresores de aire,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total,Refrigeración L1L3L4,Refrigeración L2,Prod. Linea 1(LTS),Prod. Linea 2(LTS),Prod. Linea 3(LTS),Prod. Linea 4(LTS)
0,2019-01-01,1214.4367,1281.9504,87.7999,167.8932,624.6109,1019.0041,28.3890,640512,0.0,0,399168
1,2019-01-02,4610.1620,4391.1099,883.5941,843.6383,3473.6623,5406.4273,453.9931,820448,204508.8,326400,1150848
2,2019-01-03,4646.8085,3345.8780,1850.9111,2477.3533,2907.2174,5096.9409,2516.4700,1108224,690508.8,799360,1114560
3,2019-01-04,5128.9255,2959.1534,2102.0838,2815.7259,3600.7994,5433.8828,3000.9898,533504,685843.2,577920,995328
4,2019-01-05,4565.3341,3393.3607,465.6202,2394.8061,3003.6845,5007.7196,1241.6444,442880,65318.4,335360,644112
5,2019-01-06,1876.6186,1876.0512,139.1685,198.3137,806.4864,1566.0511,214.8206,916992,0.0,0,393984
6,2019-01-07,4522.6533,3670.3405,1466.0789,1679.4776,3308.1581,5239.1975,2222.4550,631680,713836.8,721920,1136592
7,2019-01-08,4450.8017,409.9689,1706.9276,2515.3073,3200.8524,2073.4567,2327.6632,0,392688.0,162560,1101600
8,2019-01-09,3180.9719,2282.0332,28.8108,1087.5534,3075.5556,1765.4225,599.6925,600192,0.0,0,946080
9,2019-01-10,3690.3769,3637.7239,528.7063,2183.8896,3014.3048,5386.3640,497.1724,949248,309484.8,898560,1140480


## Limpieza de datos